# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.72,98,100,13.59,TF,1701204300
1,1,blackmans bay,-43.0167,147.3167,12.07,96,85,3.23,AU,1701204302
2,2,colon,9.3592,-79.9014,25.51,83,71,4.47,PA,1701204304
3,3,waitangi,-43.9535,-176.5597,15.46,90,91,2.68,NZ,1701204306
4,4,papatowai,-46.5619,169.4708,10.08,57,92,5.88,NZ,1701204307


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather = city_data_df[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 16) & (city_data_df["Humidity"] >= 40) & (city_data_df["Humidity"] <= 60)]

# Drop any rows with null values
best_weather = best_weather.dropna()

# Display sample data
best_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,127,nahiyat al fuhud,30.9697,46.7228,17.20,46,94,5.41,IQ,1701204515
161,161,san julian,21.0167,-102.1667,20.80,41,100,2.13,MX,1701204571
167,167,laguna,38.4210,-121.4238,18.99,40,0,2.06,US,1701204583
177,177,chipata,-13.6333,32.6500,23.55,57,53,3.18,ZM,1701204602
200,200,bam,29.1060,58.3570,16.62,50,3,2.13,IR,1701204645
201,201,rancho palos verdes,33.7445,-118.3870,23.88,48,0,3.60,US,1701204647
219,219,laascaanood,8.4774,47.3597,22.79,54,98,2.50,SO,1701204676
273,273,fada,9.5667,8.3000,23.25,51,2,0.30,NG,1701204774
279,279,as samawah,31.3320,45.2944,16.51,41,82,5.28,IQ,1701204785
285,285,teofilo otoni,-17.8575,-41.5053,29.50,45,72,3.09,BR,1701204795


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
127,nahiyat al fuhud,IQ,30.9697,46.7228,46,
161,san julian,MX,21.0167,-102.1667,41,
167,laguna,US,38.4210,-121.4238,40,
177,chipata,ZM,-13.6333,32.6500,57,
200,bam,IR,29.1060,58.3570,50,
201,rancho palos verdes,US,33.7445,-118.3870,48,
219,laascaanood,SO,8.4774,47.3597,54,
273,fada,NG,9.5667,8.3000,51,
279,as samawah,IQ,31.3320,45.2944,41,
285,teofilo otoni,BR,-17.8575,-41.5053,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel

radius = 10000
categories = "accommodation.hotel"


params = {
    "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nahiyat al fuhud - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
chipata - nearest hotel: Kamocho Gueshouse
bam - nearest hotel: هتل جهانگردی بم
rancho palos verdes - nearest hotel: No hotel found
laascaanood - nearest hotel: Warsame Hotel mnesta
fada - nearest hotel: No hotel found
as samawah - nearest hotel: فندق قصر الغدير
teofilo otoni - nearest hotel: Hotel Lancaster
dolores - nearest hotel: Plaza Hotel
galveston - nearest hotel: Comfort Inn & Suites Texas City
alice springs - nearest hotel: Aurora Alice Springs
ankazoabo - nearest hotel: No hotel found
viedma - nearest hotel: Níjar
pacific grove - nearest hotel: Pacific Grove Inn
machali - nearest hotel: Hotel Boutique Il Giardino
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
mulayjah - nearest hotel: No hotel found
makary - nearest hotel: No hotel found
la union - nearest hotel: No hotel found
gisborne - 

,City,Country,Lat,Lng,Humidity,Hotel Name
127,nahiyat al fuhud,IQ,30.9697,46.7228,46,No hotel found
161,san julian,MX,21.0167,-102.1667,41,No hotel found
167,laguna,US,38.4210,-121.4238,40,Holiday Inn Express & Suites
177,chipata,ZM,-13.6333,32.6500,57,Kamocho Gueshouse
200,bam,IR,29.1060,58.3570,50,هتل جهانگردی بم
201,rancho palos verdes,US,33.7445,-118.3870,48,No hotel found
219,laascaanood,SO,8.4774,47.3597,54,Warsame Hotel mnesta
273,fada,NG,9.5667,8.3000,51,No hotel found
279,as samawah,IQ,31.3320,45.2944,41,فندق قصر الغدير
285,teofilo otoni,BR,-17.8575,-41.5053,45,Hotel Lancaster


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    hover_cols=["Country", "Hotel Name"]

)
# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel Name)